### Ayudantía 5 Métodos iterativos

1.Programe los métodos de SOR, Jacobi, Gauss-Seidel y Richardson. Compare las implementaciones por entrada versus matricial.

Naive implementations

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import PIL

In [4]:
def GaussSeidel(A,b,xstart,tol=1e-5): # A = L + D + E 
    print("Gauss Seidel")
    x = xstart.copy()
    L = np.tril(A)
    U = A - L
    while abs(A@x-b).max() > tol:
        x = np.linalg.inv(L)@(b-U@x)
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
        
def SOR(A,b,xstart,w,tol=1e-5):
    print("SOR")
    x = xstart.copy()
    D = np.eye(A.shape[0])*A
    L = np.tril(A-D)
    U = np.triu(A-D)
    while abs(A@x-b).max() > tol:
        x = np.linalg.inv(D+w*L) @ ((1-w)*D@x-w*U@x+w*b)
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

def Jacobi(A,b,xstart,tol=1e-5):
    print("Jacobi")
    x = xstart.copy()
    D = np.eye(A.shape[0])*A
    L = np.tril(A-D)
    U = np.triu(A-D)
    while abs(A@x-b).max() > tol:
        x_new = np.linalg.inv(D)@(b-(L+U)@x)
        x = x_new.copy()
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

def Richardson(A,b,xstart,w,tol=1e-5):
    """inf 0.5*||Ax-b||^2 x """
    """ Looks a lot like gradient descent """
    print("Richardson")
    x = xstart.copy()
    while abs(A@x-b).max() > tol:
        x = x + w*(b-A@x) ## line search
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
def gradient_descent_exact_line_search(A,b,xstart,tol=1e-5):
    print("Gradient descent")
    x = xstart.copy()
    while abs(A@x-b).max() > tol:
        g_c = A@x-b
        mu = np.dot(g_c,g_c)/np.dot(g_c,A@g_c)
        x = x - mu*(A@x-b)
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
# w_sor = 2/(1 + np.sqrt((2*(1-rhoJ))))
def SSOR(A, w):
    D = np.diag(A.diagonal())
    Dm1 = np.diag(1/A.diagonal())
    L = np.tril(A, k=-1)
    M = (w/(2-w))*((1/w)*D + L) @ Dm1 @ ((1/w)*D + L).T
    N = M-A
    return M, N # noten que aquí se arman las matrices, queda para uds implementar la iteración


A nadie le gusta invertir matrices!🤮🤮🤮🤧🤧🤧🤯🤯🤯 

2. Pruebe los métodos de 1. con la matriz de diferencias finitas e implemente una versión más eficiente para el problema particular.

In [5]:
n = 15
test_1D = np.diag(np.ones(n)*2) + np.diag(np.ones(n-1)*-1,1) + np.diag(np.ones(n-1)*-1,-1)
test_2D = np.kron(test_1D,np.eye(n)) + np.kron(np.eye(n),test_1D)
# Define the "true solution"
x_true_1D = np.random.rand(n)
x_true_2D = np.random.rand(n**2)
# Get the right hand side
b_1D = test_1D@x_true_1D
b_2D = test_2D@x_true_2D
# Initial guess
xstart_1D = np.ones(n)
xstart_2D = np.ones(n**2)

In [6]:
# Solve the system 1D tests
t1 = time.time()
x_1D_GS = GaussSeidel(test_1D,b_1D,xstart_1D)
x_1D_SOR = SOR(test_1D,b_1D,xstart_1D,1.5)
x_1D_Jacobi = Jacobi(test_1D,b_1D,xstart_1D)
x_1D_Richardson = Richardson(test_1D,b_1D,xstart_1D,0.1)
x_1D_gradient = gradient_descent_exact_line_search(test_1D,b_1D,xstart_1D)
# x_1D_SSOR = SSOR(test_1D,b_1D,xstart_1D,1.5)
t2 = time.time()
print("Time testing 1D",t2-t1, "s")
# SSOR is useful as a preconditioner, not as a solver!

# Solve the system 2D tests
t1 = time.time()
x_2D_GS = GaussSeidel(test_2D,b_2D,xstart_2D)
x_2D_SOR = SOR(test_2D,b_2D,xstart_2D,1.5)
x_2D_Jacobi = Jacobi(test_2D,b_2D,xstart_2D)
x_2D_Richardson = Richardson(test_2D,b_2D,xstart_2D,0.1)
x_2D_SSOR = SSOR(test_2D,b_2D,xstart_2D,1.5)
t2 = time.time()
print("Time testing 2D",t2-t1, "s")



Gauss Seidel
Residual norm 2.7692996173405286e-05
SOR
Residual norm 2.3827875011993093e-05
Jacobi
Residual norm 2.5763690558533927e-05
Richardson
Residual norm 2.8274198017857964e-05
Gradient descent
Residual norm 2.0219727838028566e-05
Time testing 1D 0.13443708419799805 s
Gauss Seidel
Residual norm 7.908501461849297e-05
SOR
Residual norm 7.496311984648916e-05
Jacobi
Residual norm 6.540292628660751e-05
Richardson
Residual norm 7.940773788643053e-05
SSOR
Did not converge
Residual norm 276.7151932602389
Time testing 2D 10.483360052108765 s


In [7]:
# Not AS naive implementation
# Refer to https://sites.pitt.edu/~sts106/MEMS1055_Project_1.pdf

def GaussSeidel_byrow(A,b,xstart,tol=1e-5):
    print("Gauss Seidel by row")
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A 
    while abs(A@x-b).max() > tol:
        for i in range(A.shape[0]):
            x[i] = (1/A[i,i]) * (b[i] - np.dot(A_aux[i], x)) # np.sum([A_aux[i,j]*x[j] for j in range(A.shape[0])
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
def Jacobi_byrow(A,b,xstart,tol=1e-5): 
    print("Jacobi by row")
    x = xstart.copy()
    D = np.eye(A.shape[0])*A
    A_aux = A - D
    while abs(A@x-b).max() > tol:
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            x_new[i] = (1/A[i,i]) * (b[i] - np.dot((A_aux)[:,i],x)) 
        x = x_new.copy()
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

def SOR_byrow(A,b,xstart,w,tol=1e-5):
    print("SOR by row")
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A
    while abs(A@x-b).max() > tol:
        for i in range(A.shape[0]):
            x[i] = (1-w)*x[i] + w/A[i,i]*(b[i]- np.dot(A_aux[i], x)) #room for improvement
    print("Residual norm",np.linalg.norm(A@x-b))
    return x

In [8]:
#test new implementation
t1 = time.time()
x_1D_GS = GaussSeidel_byrow(test_1D,b_1D,xstart_1D)
x_1D_SOR = SOR_byrow(test_1D,b_1D,xstart_1D,1.5)
x_1D_Jacobi = Jacobi_byrow(test_1D,b_1D,xstart_1D)
t2 = time.time()
print("Time testing 1D by row",t2-t1, "s")

t1 = time.time()
x_2D_GS = GaussSeidel_byrow(test_2D,b_2D,xstart_2D)
x_2D_SOR = SOR_byrow(test_2D,b_2D,xstart_2D,1.5)
x_2D_Jacobi = Jacobi_byrow(test_2D,b_2D,xstart_2D)
t2 = time.time()
print("Time testing 2D by row",t2-t1, "s")



Gauss Seidel by row
Residual norm 2.7692996173442806e-05
SOR by row
Residual norm 2.3827875011993567e-05
Jacobi by row
Residual norm 2.5763690558533927e-05
Time testing 1D by row 0.018182754516601562 s
Gauss Seidel by row
Residual norm 7.908501461865292e-05
SOR by row
Residual norm 7.496311984612386e-05
Jacobi by row
Residual norm 6.54029262904003e-05
Time testing 2D by row 0.32709622383117676 s


In [9]:
# test new implementations!!!!
t1 = time.time()
x_1D_GS = GaussSeidel_byrow(test_1D,b_1D,xstart_1D)
x_1D_Jacobi = Jacobi_byrow(test_1D,b_1D,xstart_1D)
x_1D_SOR = SOR_byrow(test_1D,b_1D,xstart_1D,1.5)
t2 = time.time()
print("Time testing 1D by row",t2-t1, "s")
t1 = time.time()
x_2D_GS = GaussSeidel_byrow(test_2D,b_2D,xstart_2D)
x_2D_Jacobi = Jacobi_byrow(test_2D,b_2D,xstart_2D)
x_2D_SOR = SOR_byrow(test_2D,b_2D,xstart_2D,1.5)
t2 = time.time()
print("Time testing 2D by row",t2-t1, "s")

Gauss Seidel by row
Residual norm 2.7692996173442806e-05
Jacobi by row
Residual norm 2.5763690558533927e-05
SOR by row
Residual norm 2.3827875011993567e-05
Time testing 1D by row 0.024434328079223633 s
Gauss Seidel by row
Residual norm 7.908501461865292e-05
Jacobi by row
Residual norm 6.54029262904003e-05
SOR by row
Residual norm 7.496311984612386e-05
Time testing 2D by row 0.33122873306274414 s


<img src="img.jpg" alt="??????" />

Surely can be faster tho...

In [10]:


def SOR_finite_dif_1D(A,b,xstart,w,tol=1e-5):
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A
    while abs(A@x-b).max() > tol:
        for i in range(A.shape[0]):
            if i != 0 and i != A.shape[0]-1:
                x[i] = (1-w)*x[i] + w/A[i,i]*(b[i]- (-1*x[i-1] -1*x[i+1])) 
                continue
            elif i == 0:
                x[i] = (1-w)*x[i] + w/A[i,i]*(b[i]- (-1*x[1]))
            elif i == A.shape[0]-1:
                x[i] = (1-w)*x[i] + w/A[i,i]*(b[i]- (-1*x[A.shape[0]-2]))
            
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
def jacobi_finite_dif_1D(A,b,xstart,tol=1e-5):

    x = xstart.copy()
    D = np.eye(A.shape[0])*A
    A_aux = A - D
    while abs(A@x-b).max() > tol:
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            if i != 0 and i != A.shape[0]-1:
                x_new[i] = (1/A[i,i]) * (b[i] - (-1*x[i-1] -1*x[i+1]))
                continue
            elif i == 0:
                x_new[i] = (1/A[i,i]) * (b[i] - (-1*x[1])) # why does this work??
            elif i == A.shape[0]-1:
                x_new[i] = (1/A[i,i]) * (b[i] - (-1*x[A.shape[0]-2]))
        x = x_new.copy()
            
    print("Residual norm",np.linalg.norm(A@x-b))
    return x
def GaussSeidel_finite_dif_1D(A,b,xstart,tol=1e-5):
    
    x = xstart.copy()
    A_aux = A - np.eye(A.shape[0])*A
    while abs(A@x-b).max() > tol:
        
        for i in range(A.shape[0]):
            if i != 0 and i != A.shape[0]-1:
                x[i] = (1/A[i,i]) * (b[i] - (-1*x[i-1] -1*x[i+1]) )
                continue
            elif i == 0:
                x[i] = (1/A[i,i]) * (b[i] - (-1*x[1])) 
            elif i == A.shape[0]-1:
                x[i] = (1/A[i,i]) * (b[i] - (-1*x[A.shape[0]-2]))
    print("Residual norm",np.linalg.norm(A@x-b))
    return x



In [11]:
# big n 1D super naive, not as naive and specially made for the problem side by side
n = 200
test_1D = np.diag(np.ones(n)*2) + np.diag(np.ones(n-1)*-1,1) + np.diag(np.ones(n-1)*-1,-1)
b_1D = np.random.rand(n)
xstart_1D = np.ones(n)

In [12]:
print("Testing naive")
t1 = time.time()
x_1D_GS = GaussSeidel(test_1D,b_1D,xstart_1D)
x_1D_SOR = SOR(test_1D,b_1D,xstart_1D,1.5)
x_1D_Jacobi = Jacobi(test_1D,b_1D,xstart_1D)
t2 = time.time()
print("Time testing 1D naive",t2-t1, "s")
print()
print("Testing not as naive")
t1 = time.time()
x_1D_GS = GaussSeidel_byrow(test_1D,b_1D,xstart_1D)
x_1D_Jacobi = Jacobi_byrow(test_1D,b_1D,xstart_1D)
x_1D_SOR = SOR_byrow(test_1D,b_1D,xstart_1D,1.5)
t2 = time.time()
print("Time testing 1D not as naive",t2-t1, "s")
print()
print("Testing specially made for the problem")
t1 = time.time()
x_1D_SOR = SOR_finite_dif_1D(test_1D,b_1D,xstart_1D,1.5)
x_1D_Jacobi = jacobian_finite_dif_1D(test_1D,b_1D,xstart_1D)
x_1D_GS = GaussSeidel_finite_dif_1D(test_1D,b_1D,xstart_1D)
t2 = time.time()
print("Time testing 1D specially made",t2-t1, "s")

Testing naive
Gauss Seidel
Residual norm 0.00010024609998168708
SOR
Residual norm 0.00010020743347058732
Jacobi
Residual norm 0.00010020280307993888
Time testing 1D naive 149.85855388641357 s

Testing not as naive
Gauss Seidel by row
Residual norm 0.00010024609960032224
Jacobi by row
Residual norm 0.00010020280307993888
SOR by row
Residual norm 0.00010020743345076767
Time testing 1D not as naive 36.68470501899719 s

Testing specially made for the problem
Residual norm 0.00010020743345076767


NameError: name 'jacobian_finite_dif_1D' is not defined

<img src="img2.jpeg" alt="??????" />